File: 6.3_Medeiros_Midterm_NYC_Airbnb.ipynb  
Name: Corinne Medeiros  
Date: 01/19/20  
Course: DSC 540 Data Preparation  
Desc: 6.3 Midterm Project - New York City Airbnb Data  
Usage: Program reads, cleans, formats, and investigates data from NYC Airbnb csv file.

## Midterm Project  
#### New York City Airbnb Data from <https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data>

In [3]:
import csv

import pandas as pd

import numpy as np

from fuzzywuzzy import fuzz

import datetime

In [4]:
# Importing csv file, converting 'last_review' column to datetime
nyc_airbnb_df = pd.read_csv("AB_NYC_2019.csv", parse_dates=['last_review'])

In [5]:
# Previewing data
nyc_airbnb_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaT,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [6]:
# Summary of dataframe
print(nyc_airbnb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
id                                48895 non-null int64
name                              48879 non-null object
host_id                           48895 non-null int64
host_name                         48874 non-null object
neighbourhood_group               48895 non-null object
neighbourhood                     48895 non-null object
latitude                          48895 non-null float64
longitude                         48895 non-null float64
room_type                         48895 non-null object
price                             48895 non-null int64
minimum_nights                    48895 non-null int64
number_of_reviews                 48895 non-null int64
last_review                       38843 non-null datetime64[ns]
reviews_per_month                 38843 non-null float64
calculated_host_listings_count    48895 non-null int64
availability_365                  48895 non-nu

### Replacing Headers

In this dataset, the headers are very readable, so I decided to replace them with capitalized versions for practice and demonstration purposes.  

First, I copied the header names from the original csv file into a new csv file with their corresponding new headers.

In [7]:
from csv import reader

data_rdr = reader(open('AB_NYC_2019.csv', 'rt'))
header_rdr = reader(open('AB_NYC_2019_headers.csv', 'rt'))

# Creating list from each row of file imported into data_rdr
data_rows = [d for d in data_rdr]

# Creating list from each row of file imported into header_rdr,
# except when first item of header_rdr does not exist within data_rows headers.
header_rows = [h for h in header_rdr if h[0] in data_rows[0]] 

# Creating list of old headers
old_headers = [h[0] for h in header_rows]

skip_index = []
final_header_rows = []

# Creating list of indexes for rows in which header from data_rows is not in old_headers.
# Also creating a list of header rows in the correct order.
for header in data_rows[0]:
    if header not in old_headers:
        index = data_rows[0].index(header)
        skip_index.append(index) 
    else:
        for head in header_rows: 
            if head[0] == header:
                final_header_rows.append(head) 
                break
                
new_data = []

# Creating ordered list of data to match up with each item in final_header_rows list.
# Skipping rows in skip_index list.
for row in data_rows[1:]:
    new_row = []
    for i, d in enumerate(row): 
        if i not in skip_index:
            new_row.append(d)
    new_data.append(new_row)

zipped_data = []

# Putting new_data and final_header_rows into tuples.
# Each row includes all corresponding data with headers.
for drow in new_data:
    zipped_data.append(list(zip(final_header_rows, drow)))
    
zipped_data[0]

[(['id', 'ID'], '2539'),
 (['name', 'NAME'], 'Clean & quiet apt home by the park'),
 (['host_id', 'HOST_ID'], '2787'),
 (['host_name', 'HOST_NAME'], 'John'),
 (['neighbourhood_group', 'NEIGHBORHOOD_GROUP'], 'Brooklyn'),
 (['neighbourhood', 'NEIGHBORHOOD'], 'Kensington'),
 (['latitude', 'LATITUDE'], '40.64749'),
 (['longitude', 'LONGITUDE'], '-73.97237'),
 (['room_type', 'ROOM_TYPE'], 'Private room'),
 (['price', 'PRICE'], '149'),
 (['minimum_nights', 'MINIMUM_NIGHTS'], '1'),
 (['number_of_reviews', 'NUMBER_OF_REVIEWS'], '9'),
 (['last_review', 'LAST_REVIEW'], '2018-10-19'),
 (['reviews_per_month', 'REVIEWS_PER_MONTH'], '0.21'),
 (['calculated_host_listings_count', 'CALCULATED_HOST_LISTINGS_COUNT'], '6'),
 (['availability_365', 'AVAILABILITY_365'], '365')]

### Formatting Data to a Readable Format

In [8]:
# Looping through first row of data, printing header and corresponding value on separate lines
for x in zipped_data[0]:
    print ('Header: {[1]}\nValue: {}'.format(x[0], x[1]))

Header: ID
Value: 2539
Header: NAME
Value: Clean & quiet apt home by the park
Header: HOST_ID
Value: 2787
Header: HOST_NAME
Value: John
Header: NEIGHBORHOOD_GROUP
Value: Brooklyn
Header: NEIGHBORHOOD
Value: Kensington
Header: LATITUDE
Value: 40.64749
Header: LONGITUDE
Value: -73.97237
Header: ROOM_TYPE
Value: Private room
Header: PRICE
Value: 149
Header: MINIMUM_NIGHTS
Value: 1
Header: NUMBER_OF_REVIEWS
Value: 9
Header: LAST_REVIEW
Value: 2018-10-19
Header: REVIEWS_PER_MONTH
Value: 0.21
Header: CALCULATED_HOST_LISTINGS_COUNT
Value: 6
Header: AVAILABILITY_365
Value: 365


### Identifying Outliers and Bad Data

#### Looking for missing values

In [9]:
# Checking for null values in original dataframe
nyc_airbnb_df.isnull().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [10]:
# Empty dictionary to store NA values
na_count = {}

# Looping through zipped_data rows, checking for NA values
for row in zipped_data:
    for item in row:
        header = item[0][1] 
        value = item[1]
        if value in ["NA", "na", "n/a", "", None]:
            if header in na_count.keys(): 
                na_count[header] += 1 
            else:
                na_count[header] = 1 

print (na_count)

{'LAST_REVIEW': 10052, 'REVIEWS_PER_MONTH': 10052, 'HOST_NAME': 21, 'NAME': 16}


Using pandas and loops, we can confirm that the dataset contains missing values. Knowing that it is possible for Airbnb listings to have 0 reviews, with a matching value for reviews per month tells us that these are most likely not bad data. Name and host name on the other hand are columns that should have data, so removing these could be a good idea, but would need more research into those individual listings to determine the quality.

One example of dealing with missing values would be replacing some null values with 0. Since blank fields within the reviews per month column most likely mean there aren't any reviews for that listing, I could put 0 instead.

In [11]:
# Replacing null values in reviews per month with 0 
nyc_airbnb_df["reviews_per_month"].fillna(0, inplace = True)

In [12]:
# Previewing data after replacing null values
nyc_airbnb_df['reviews_per_month'].head()

0    0.21
1    0.38
2    0.00
3    4.64
4    0.10
Name: reviews_per_month, dtype: float64

#### Looking for Outliers

In [13]:
# Verifing that fields are the expected data type by looping through the data and storing results in a dictionary

datatypes = {} 

# Creating dictionary of different possible data types
start_dict = {'digit': 0, 'boolean': 0,
              'empty': 0, 'time_related': 0,
              'text': 0, 'unknown': 0
              } 

# Looping through data, performing initial type checks, storing values in corresponding type key
for row in zipped_data:
    for item in row:
        header = item[0][1]
        value = item[1]
        key = 'unknown' 
        if value.isdigit(): 
            key = 'digit'
        elif value in ['Yes', 'No', 'True', 'False']: 
            key = 'boolean'
        elif value.isspace(): 
            key = 'empty'
        elif value.find('/') > 0 or value.find(':') > 0: 
            key = 'time_related'
        elif value.isalpha(): 
            key = 'text'
        if header not in datatypes.keys(): 
            datatypes[header] = start_dict.copy() 
        datatypes[header][key] += 1 

print(datatypes)


{'ID': {'digit': 48895, 'boolean': 0, 'empty': 0, 'time_related': 0, 'text': 0, 'unknown': 0}, 'NAME': {'digit': 4, 'boolean': 0, 'empty': 0, 'time_related': 5102, 'text': 186, 'unknown': 43603}, 'HOST_ID': {'digit': 48895, 'boolean': 0, 'empty': 0, 'time_related': 0, 'text': 0, 'unknown': 0}, 'HOST_NAME': {'digit': 6, 'boolean': 0, 'empty': 0, 'time_related': 42, 'text': 44929, 'unknown': 3918}, 'NEIGHBORHOOD_GROUP': {'digit': 0, 'boolean': 0, 'empty': 0, 'time_related': 0, 'text': 48522, 'unknown': 373}, 'NEIGHBORHOOD': {'digit': 0, 'boolean': 0, 'empty': 0, 'time_related': 0, 'text': 20551, 'unknown': 28344}, 'LATITUDE': {'digit': 0, 'boolean': 0, 'empty': 0, 'time_related': 0, 'text': 0, 'unknown': 48895}, 'LONGITUDE': {'digit': 0, 'boolean': 0, 'empty': 0, 'time_related': 0, 'text': 0, 'unknown': 48895}, 'ROOM_TYPE': {'digit': 0, 'boolean': 0, 'empty': 0, 'time_related': 25409, 'text': 0, 'unknown': 23486}, 'PRICE': {'digit': 48895, 'boolean': 0, 'empty': 0, 'time_related': 0, 'te

In [14]:
# Printing datatypes results in an easier to read format
for key in datatypes:
    print ('Header: {}\nData Types: {}\n'.format(key, datatypes[key]))

Header: ID
Data Types: {'digit': 48895, 'boolean': 0, 'empty': 0, 'time_related': 0, 'text': 0, 'unknown': 0}

Header: NAME
Data Types: {'digit': 4, 'boolean': 0, 'empty': 0, 'time_related': 5102, 'text': 186, 'unknown': 43603}

Header: HOST_ID
Data Types: {'digit': 48895, 'boolean': 0, 'empty': 0, 'time_related': 0, 'text': 0, 'unknown': 0}

Header: HOST_NAME
Data Types: {'digit': 6, 'boolean': 0, 'empty': 0, 'time_related': 42, 'text': 44929, 'unknown': 3918}

Header: NEIGHBORHOOD_GROUP
Data Types: {'digit': 0, 'boolean': 0, 'empty': 0, 'time_related': 0, 'text': 48522, 'unknown': 373}

Header: NEIGHBORHOOD
Data Types: {'digit': 0, 'boolean': 0, 'empty': 0, 'time_related': 0, 'text': 20551, 'unknown': 28344}

Header: LATITUDE
Data Types: {'digit': 0, 'boolean': 0, 'empty': 0, 'time_related': 0, 'text': 0, 'unknown': 48895}

Header: LONGITUDE
Data Types: {'digit': 0, 'boolean': 0, 'empty': 0, 'time_related': 0, 'text': 0, 'unknown': 48895}

Header: ROOM_TYPE
Data Types: {'digit': 0, '

From these results, we can see that certain values might need to be checked, such as latitude, longitude, reviews per month, and names of places. If we needed to run statistical tests using reviews per month for instance, we would need to make sure the values grouped under 'unknown' are recognized as numbers, or replaced.

### Finding Duplicates

In [16]:
# Selecting duplicate rows based on 'name' & 'host_id' columns using Pandas
nyc_df_duplicates_2 = nyc_airbnb_df[nyc_airbnb_df.duplicated(['name', 'host_id'])]
print("Duplicate rows based on 'name' & 'host_id' columns: ", nyc_df_duplicates_2, sep='\n')

Duplicate rows based on 'name' & 'host_id' columns: 
             id                                               name    host_id  \
330       81739                  Loft w/ Terrace @ Box House Hotel     417504   
339       84010                               Superior @ Box House     417504   
580      219818                 ✿✿✿ COUNTRY COTTAGE IN THE CITY✿✿✿    1138692   
661      250537                                The Lenox in Harlem    1313306   
669      253471                   Loft Suite @ The Box House Hotel     417504   
670      253475                   Loft Suite @ The Box House Hotel     417504   
674      253800                       Loft Suite @ Box House Hotel     417504   
675      253803                   Loft Suite @ The Box House Hotel     417504   
676      253806                   Loft Suite @ The Box House Hotel     417504   
677      253811                   Loft Suite @ The Box House Hotel     417504   
678      253815                   Loft Suite @ The Box H

Although there seem to be a lot of duplicate listings using the name and host id columns, I would hesitate to remove these duplicates because it could be possible that the same host owns multiple properties with the same name. In order to determine which listings to remove, I could try matching up more than these two columns.

### Fuzzy Matching

In [17]:
# Converting listing names column to a list
names_list = nyc_airbnb_df['name'].tolist()

In [18]:
# Creating a subset containing only the first twenty listing names
first_twenty_names = names_list[0:20]

# Previewing first twenty names
for item in first_twenty_names:
    print (item)

Clean & quiet apt home by the park
Skylit Midtown Castle
THE VILLAGE OF HARLEM....NEW YORK !
Cozy Entire Floor of Brownstone
Entire Apt: Spacious Studio/Loft by central park
Large Cozy 1 BR Apartment In Midtown East
BlissArtsSpace!
Large Furnished Room Near B'way 
Cozy Clean Guest Room - Family Apt
Cute & Cozy Lower East Side 1 bdrm
Beautiful 1br on Upper West Side
Central Manhattan/near Broadway
Lovely Room 1, Garden, Best Area, Legal rental
Wonderful Guest Bedroom in Manhattan for SINGLES
West Village Nest - Superhost
Only 2 stops to Manhattan studio
Perfect for Your Parents + Garden
Chelsea Perfect
Hip Historic Brownstone Apartment with Backyard
Huge 2 BR Upper East  Cental Park


In [19]:
from fuzzywuzzy import process

# Comparing string 'Cozy Apartment' with first twenty listing names using extract, returns two possible matches
process.extract('Cozy Apartment', first_twenty_names, limit=2)

[('Cozy Entire Floor of Brownstone', 86),
 ('Large Cozy 1 BR Apartment In Midtown East', 86)]

In [20]:
# Finding only the best match using the extractOne method
process.extractOne('Cozy Apartment', first_twenty_names)

('Cozy Entire Floor of Brownstone', 86)

In [21]:
# Comparing string 'Park' with first twenty listing names using extract, returns two possible matches
process.extract('Park', first_twenty_names, limit=2)

[('Clean & quiet apt home by the park', 60),
 ('Entire Apt: Spacious Studio/Loft by central park', 60)]

In [22]:
# Finding only the best match using the extractOne method
process.extractOne('Park', first_twenty_names)

('Clean & quiet apt home by the park', 60)

Using fuzzywuzzy's `process` module, we can `extract` results from our list of names that are similar to the chosen string. Our output returns tuples containing potential matches with the similarity ratios, so we can then compare how similar or different the matches are. The `extractOne` function finds only the best match, which can be useful when you're only interested in the top result.